## Models

In [1]:
from dotenv import load_dotenv  
import os    
from langchain_openai import AzureChatOpenAI,AzureOpenAIEmbeddings

# Load the .env file  
load_dotenv()  
  # Get an environment variable  
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT') 
key = os.getenv('AZURE_OPENAI_API_KEY')
api_version = os.getenv('AZURE_OPENAI_API_VERSION')  
chat_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT')  
embedding_deployment = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT')

chat_model = AzureChatOpenAI(
    azure_deployment=chat_deployment,
    openai_api_version=api_version,
    temperature=0
    )
embedding_model=AzureOpenAIEmbeddings(
    azure_deployment=embedding_deployment,
    openai_api_version=api_version,)

# Tools

In [2]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.agents import AgentType, initialize_agent
from pptx import Presentation
import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
import bs4
import json

def read_ppt_withidx(source_path: str) -> tuple[object, list]:
    '''Read a PowerPoint file and return the file and the text content of each slide with index of slide, shape and paragraph.
    Args:  source_path: The PowerPoint file path'''
    try:
        print(1)
        pptfile = Presentation(source_path)
        slides_text = []
        for page_idx in range(len(pptfile.slides)):
            slide_text = []
            for shape_idx in range(len(pptfile.slides[page_idx].shapes)):
                if hasattr(pptfile.slides[page_idx].shapes[shape_idx], "text") and pptfile.slides[page_idx].shapes[shape_idx].has_text_frame:
                    for idx in range(len(pptfile.slides[page_idx].shapes[shape_idx].text_frame.paragraphs)):
                        run_text = pptfile.slides[page_idx].shapes[shape_idx].text_frame.paragraphs[idx].runs
                        for run in run_text:
                            slide_text.append([page_idx,shape_idx,idx,run.text])
            slides_text.append(slide_text)
    except:
        return 'File not found!'
    print(slides_text)
    return slides_text

def correct_words_withidx(source_path:str,save_path:str,replace_dict:dict)-> str:
    '''Update the powerpoint file load before with the correct words, and save it to a new file. 
    Only when the Input Plan is asked to replace the word and user approved 
    save_path:The new PowerPoint file path.
    replace_dict:The dict of words need to be replaced, key is the word to be replaced, value is a list of the index of the slide, shape and paragraph and the new word.
    '''
    pptfile = Presentation(source_path)
    for search_str, repl_str in replace_dict.items():
        if len(repl_str)==1:
            repl_str=repl_str[0]
        for x in range(len(pptfile.slides)):
            for s in range(len(pptfile.slides[x].shapes)):
                if hasattr(pptfile.slides[x].shapes[s], "text") and pptfile.slides[x].shapes[s].has_text_frame:
                    for idx in range(len(pptfile.slides[x].shapes[s].text_frame.paragraphs)):
                        run_text = pptfile.slides[x].shapes[s].text_frame.paragraphs[idx].runs
                        for run in run_text:
                            if(run.text.find(search_str))!=-1 and x==repl_str[0] and s==repl_str[1] and idx==repl_str[2]:
                                run.text = run.text.replace(str(search_str), str(repl_str[3]))
    pptfile.save(save_path)
    return 'Save succeed!'

def retriever(query:str) -> list:
    '''Load external knowledge base to help understand question and files. 
    Arg: query: Question to get the definition from the external knowledge base."
    '''
    directory="./ExternalKnowledge"
    if os.path.exists(f"{directory}/chroma.sqlite3"):  
        vectorstore = Chroma(persist_directory=directory, embedding_function=embedding_model)
    else:  
        print(f"Index does not exist at {directory}, building") 
        # local files
        pdf_files = glob.glob(f"{directory}/*.pdf")
        all_docs = []
        for pdf_file in pdf_files:
            loader = PyPDFLoader(pdf_file)
            all_docs.extend(loader.load())
        with open(f"{directory}/websites.json", 'r', encoding='utf-8') as file:
            contents = json.load(file)
        loader = WebBaseLoader(
            web_paths=(contents['Weblist']),
            requests_kwargs = {'verify':False})
        all_docs=all_docs+loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(all_docs)
        vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory=directory)
    retrieve = vectorstore.as_retriever()
    
    prompt = ChatPromptTemplate.from_messages(
    [("system", '''Use the following pieces of context to answer the question at the end, which is a terminology dictionary tell the abbreviation the meaning of the words.
        If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum and keep the answer as concise as possible.{context}'''),
    ("human", '{question}If there is a list of workds,Which are meaningful and whcih are not?'),])
    rag_chain = (
        {"context": retrieve , "question": RunnablePassthrough()}
        | prompt
        | chat_model
        | StrOutputParser()
    )
    return rag_chain.invoke(query)

read_ppt_withidx = StructuredTool.from_function(read_ppt_withidx)
correct_words_withidx = StructuredTool.from_function(correct_words_withidx)
retriever = StructuredTool.from_function(retriever)

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Prompts

In [3]:
from langchain import PromptTemplate

plan_prompt = PromptTemplate(
    input_variables=["input", "chat_history"],
    template="""You are a meticulous and detail-oriented clerk. Prepare a plan for task execution. Let's think step by step.
    Tools to use: 
    - read_ppt_withidx: Read a PowerPoint file and return the file and the text content of each slide with the index of slide, shape, and paragraph
    - retriever: Load external knowledge base to help understand questions and files
    - correct_words_withidx: Update the PowerPoint file load with the correct spelling before, and save it to a new file.
    
    Question: {input}
    History: {chat_history}
    
    Always get the list of words that meet user's requirement first before running the retriever.
    Create words list that are not meaningful but without spelling mistakes from the oberservation, and use the retriever to check if they are meaningful in the external knowledge. Then remove the meaningful words from the correction list.
    If the user asks to correct the words, always ask for the user's confirmation of the corrention list before running correct_words_withidx and use the confirmed ones to correct. Otherwise, do not plan any task to correct words.
    The output should look like this:
        User Question: {input}
        Execution plan: [execution_plan]
        Rest of needed information: [rest_of_needed_information]
    Only plan the task the user mentioned in the question and do not add extra steps.
    """,
)

prompt = PromptTemplate(
    template="""
    Plan:{input}
    History: {chat_history}
    You are an asstistant to solve or answer the user's question by executing given the plan step by step. You have to follow the user's plan based on the previous observation. 
    
    Don't finish the work until all plans are completed and finish the task when the final step is done.
    
    Use retriever to check the meaning of the words looks meaningless but do not have spelling mistakes and are not the words used wrongly.
    
    Always finish and output the word list you want to corret to ask user's confirmation and advice.
    
    When need to call correct_words_withidx, review the user question to find is there any approval or confirmation said you can save file. If yes, say I will start save the corrected file and run the correct_words_withidx with the confirmed list, if not, output the current corrention list, ask user for confirmation and finish.
    
    Do not show shape and paragraph index in the final output.
    IMPORTANT: Mustn't take any action that is not in the plan.
    """,
    input_variables=["input", "chat_history"],
    )
# When need to check if the words are meaningful, gether all the words from previous observation, which is meaningless and no spelling mistake, and ask which are meaningful in the list and which are not with retriever tool.For example, the input of retriever should be "Is DEV, UIS and Caad meaningful?".

# Agent

In [4]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

def run_agent(input_text: str) -> str:
    plan_result = plan_chain.run(input_text)
    # Agent execution
    # print(plan_result)
    res = agent(plan_result)
    return plan_result,res['output']

memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True)
# Initialize Plan Chain
plan_chain = ConversationChain(
    llm=chat_model,
    memory=memory,
    input_key="input",
    prompt=plan_prompt,
    output_key="output",
)
# Initialize Agent
agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[read_ppt_withidx,retriever,correct_words_withidx],
    llm=chat_model,
    verbose=True, # verbose option is for printing logs (only for development)
    max_iterations=10,
    prompt=prompt,
    memory=memory,
)


C:\Users\wenjieyang\AppData\Local\Temp\ipykernel_3200\449136048.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True)
C:\Users\wenjieyang\AppData\Local\Temp\ipykernel_3200\449136048.py:13: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  plan_chain = ConversationChain(
C:\Users\wenjieyang\AppData\Local\Temp\ipykernel_3200\449136048.py:21: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_ag

# Run

In [5]:
input_str = "Is there any word with spelling mistake, wrong word with similar speeling or looks like meaningless word in the ppt file with path './What is AI Agent.pptx? "
plan,response=run_agent(input_str)
response

C:\Users\wenjieyang\AppData\Local\Temp\ipykernel_3200\449136048.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  plan_result = plan_chain.run(input_text)
C:\Users\wenjieyang\AppData\Local\Temp\ipykernel_3200\449136048.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = agent(plan_result)




> Entering new AgentExecutor chain...
Understood. Let's start by reading the PowerPoint file located at './What is AI Agent.pptx' to extract the text content of each slide along with the index of slide, shape, and paragraph.

Action:
```
{
  "action": "read_ppt_withidx",
  "action_input": {
    "source_path": "./What is AI Agent.pptx"
  }
}
```1
[[[0, 6, 0, 'What is AI Agnt?'], [0, 7, 0, 'Intelligent agents that perceive environment and achieve '], [0, 7, 0, 'goats']], [[1, 4, 0, 'Agenda Items'], [1, 5, 0, 'Concept of AI Agent.'], [1, 5, 1, 'Application of AI Agent.']], [[2, 2, 0, 'Concept of AI Agent']], [[3, 5, 0, 'Definition of AI Agent'], [3, 6, 1, 'An AI agent is an intelligent entity that perceves its environment and takes actions to achieve its goals. AI agents can be designed to interact with other agents and their environment, and they can learn from their experiences.']], [[4, 2, 0, 'Types of AI Agent'], [4, 4, 0, 'Reactive Agents'], [4, 4, 1, 'Reactive agents are the simpl

⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.



Observation: Based on the provided context, "AgWf" is meaningful. It stands for "Agile Waterfall," which is a work tracking, planning, and reporting methodology used by the Devices SW team.
Thought:Next, I will use the `retriever` tool to check if the word "MSFTT" is actually meaningful in the external knowledge base.

Action:
```
{
  "action": "retriever",
  "action_input": {
    "query": "MSFTT"
  }
}
```

Observation: Based on the provided context, the following abbreviations are meaningful:

1. MTI - Manufacturing Test Infrastructure
2. MTF - Modulation Transfer Function
3. MTEOS - Windows OS image used on the manufacturing line
4. MTK - Media Test Kit (Xbox One Controller Related)
5. MTK - In the context of RF power and SAR, refers to parts of Asia with different requirements than the U.S. or European Union

Any other abbreviations not listed in the context are not meaningful based on the provided information.
Thought:Based on the observations, the word "MSFTT" is not identified 

"The PowerPoint file contains the following words with spelling mistakes, similar spelling errors, or meaningless words:\n\n1. Spelling Mistakes:\n   - Slide 0, Shape 6, Paragraph 0: 'Agnt' (should be 'Agent')\n   - Slide 3, Shape 6, Paragraph 1: 'perceves' (should be 'perceives')\n   - Slide 5, Shape 3, Paragraph 1: 'comands' (should be 'commands')\n\n2. Words with Similar Spelling Errors:\n   - Slide 0, Shape 7, Paragraph 0: 'goats' (context suggests it should be 'goals')\n\n3. Meaningless Words:\n   - Slide 7, Shape 6, Paragraph 2: 'MSFTT'\n\nThe word 'AgWf' is meaningful and should not be considered for correction."

In [6]:
print(plan)

User Question: Is there any word with spelling mistake, wrong word with similar speeling or looks like meaningless word in the ppt file with path './What is AI Agent.pptx? 

Execution plan:
1. Use the `read_ppt_withidx` tool to read the PowerPoint file located at './What is AI Agent.pptx' and extract the text content of each slide along with the index of slide, shape, and paragraph.
2. Analyze the extracted text to identify words with spelling mistakes, words with similar spelling that might be incorrect, and words that appear to be meaningless.
3. Create a list of words that are not meaningful but without spelling mistakes from the observation.
4. Use the `retriever` tool to check if the words identified as meaningless are actually meaningful in the external knowledge base.
5. Remove any words from the correction list that are found to be meaningful by the `retriever`.

Rest of needed information:
- The text content of each slide with the index of slide, shape, and paragraph.
- The li

In [7]:
input_str = "Yes, please correct the words with the content and index you found. \
            Besides,change MSFTT to MSFT. The save path is ./What is AI Agent_correct.pptx."
run_agent(input_str)




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "correct_words_withidx",
  "action_input": {
    "source_path": "./What is AI Agent.pptx",
    "save_path": "./What is AI Agent_correct.pptx",
    "replace_dict": {
      "Agnt": [[0, 6, 0, "Agent"]],
      "perceves": [[3, 6, 1, "perceives"]],
      "comands": [[5, 3, 1, "commands"]],
      "goats": [[0, 7, 0, "goals"]],
      "MSFTT": [[7, 6, 2, "MSFT"]]
    }
  }
}
```
Observation: Save succeed!
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The PowerPoint file has been successfully corrected and saved to './What is AI Agent_correct.pptx'. The following corrections were made: 'Agnt' to 'Agent', 'perceves' to 'perceives', 'comands' to 'commands', 'goats' to 'goals', and 'MSFTT' to 'MSFT'."
}
```

> Finished chain.


("User Question: Yes, please correct the words with the content and index you found. Besides, change MSFTT to MSFT. The save path is ./What is AI Agent_correct.pptx.\n\nExecution plan:\n1. Use the `correct_words_withidx` tool to update the PowerPoint file with the correct spelling for the identified words and save it to a new file.\n2. Specifically, correct the following words:\n   - Slide 0, Shape 6, Paragraph 0: 'Agnt' to 'Agent'\n   - Slide 3, Shape 6, Paragraph 1: 'perceves' to 'perceives'\n   - Slide 5, Shape 3, Paragraph 1: 'comands' to 'commands'\n   - Slide 0, Shape 7, Paragraph 0: 'goats' to 'goals'\n   - Slide 7, Shape 6, Paragraph 2: 'MSFTT' to 'MSFT'\n3. Save the corrected PowerPoint file to the specified path: `./What is AI Agent_correct.pptx`.\n\nRest of needed information:\n- The text content of each slide with the index of slide, shape, and paragraph.\n- The list of words identified as having spelling mistakes, similar spelling errors, or appearing meaningless.\n- The c